<a href="https://colab.research.google.com/github/JuanSebastianSantos/PROGCOM-A/blob/main/gaq4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import threading
import time
import random
import sys

# --- CONFIGURACIÓN DE INGREDIENTES ---
INGREDIENTES_DISPONIBLES = {
    1: "Tomate",
    2: "Mozzarella",
    3: "Pepperoni",
    4: "Champiñones",
    5: "Cebolla",
    6: "Pimiento",
    7: "Aceitunas",
    8: "Anchoas",
    9: "Piña",
}

# --- Clases del Juego ---

class Comensal(threading.Thread):
    """Representa a un cliente y su temporizador de paciencia."""
    def __init__(self, id_comensal, tolerancia, ingredientes_req, es_especial):
        super().__init__()
        self.id = id_comensal
        self.tolerancia = tolerancia
        self.ingredientes_req = ingredientes_req # Lista de ingredientes que quiere
        self.es_especial = es_especial          # Grupo 2: No puede/quiere comprar
        self.se_fue = threading.Event()         # Flag para indicar que se fue por tiempo
        self.atendido = threading.Event()       # Flag para indicar que el jugador terminó

    def run(self):
        """El hilo del cliente: cuenta el tiempo de paciencia."""

        # Espera hasta que el jugador termine O hasta que se acabe el tiempo.
        # timeout=self.tolerancia hace que espere solo ese tiempo
        if not self.atendido.wait(timeout=self.tolerancia):
            # Si el wait terminó por timeout (tiempo agotado)
            if not self.atendido.is_set():
                self.se_fue.set()
                print(f"\n🚨 {self.id} ABANDONA: ¡Excediste su tolerancia de {self.tolerancia:.2f} segundos!")

class Pizzeria:
    def __init__(self, clan_infectado):
        self.balance_caja = 0.0
        self.clientes_satisfechos = 0
        self.clientes_perdidos = 0
        self.clan_infectado = clan_infectado
        self.num_clientes = 0

        if self.clan_infectado:
            print("⚠️ CLAN INFECTADO (Grupo 5): ¡La tolerancia de los clientes es baja!")
        else:
            print("✅ CLAN SANO.")

    def generar_pedido(self):
        """Genera un pedido aleatorio (Grupo 1) y un cliente (Grupo 2 y 5)."""
        self.num_clientes += 1

        # Grupo 5: Tolerancia reducida si el clan está infectado
        tolerancia_base = random.uniform(15, 25)
        tolerancia_final = tolerancia_base / (1.5 if self.clan_infectado else 1.0)

        # Grupo 2: Clientes especiales (15% de probabilidad)
        es_especial = random.random() < 0.15

        # Grupo 1: Ingredientes requeridos
        num_ingredientes_req = random.randint(3, 5)
        ingredientes_req = random.sample(list(INGREDIENTES_DISPONIBLES.values()), num_ingredientes_req)

        cliente = Comensal(f"Cliente_{self.num_clientes}", tolerancia_final, ingredientes_req, es_especial)
        return cliente

    def mostrar_menu_ingredientes(self):
        print("\n--- MENÚ DE INGREDIENTES (Elige el número) ---")
        for num, ing in INGREDIENTES_DISPONIBLES.items():
            print(f"[{num}] {ing}")
        print("[S] ¡Servir Pizza! (Terminar pedido)")
        print("-" * 35)

    def atender_cliente(self, cliente):
        """Bucle principal de interacción con el jugador."""

        # 1. Presentar el pedido
        print("\n" + "#" * 50)
        print(f"| LLEGA: {cliente.id} | {'EVENTO ESPECIAL (G2)' if cliente.es_especial else 'CLIENTE NORMAL'}")
        print(f"| TOLERANCIA: {cliente.tolerancia:.2f} segundos")
        print(f"| QUIERE: {', '.join(cliente.ingredientes_req)}")
        print("#" * 50)

        ingredientes_hechos = []

        # 2. Iniciar el hilo del cliente (temporizador)
        cliente.start()
        tiempo_inicio = time.time()

        # 3. Bucle interactivo
        while not cliente.se_fue.is_set():
            self.mostrar_menu_ingredientes()
            print(f"🍕 Pizza actual: {', '.join(ingredientes_hechos) if ingredientes_hechos else 'Vacía'}")

            # Cálculo del tiempo restante (para mostrar al jugador)
            tiempo_transcurrido = time.time() - tiempo_inicio
            tiempo_restante = cliente.tolerancia - tiempo_transcurrido
            print(f"⏰ TIEMPO RESTANTE: {tiempo_restante:.2f} segundos")

            try:
                opcion = input(">>> Elige Ingrediente o [S]ervir: ").strip().upper()
            except EOFError:
                # Manejo de EOF en entornos como Colab/Jupyter que pueden terminar el input
                opcion = 'S'

            if opcion == 'S':
                break # Sale del bucle para verificar la pizza

            if opcion.isdigit():
                num_ing = int(opcion)
                if num_ing in INGREDIENTES_DISPONIBLES:
                    ingrediente_sel = INGREDIENTES_DISPONIBLES[num_ing]
                    if ingrediente_sel not in ingredientes_hechos:
                        ingredientes_hechos.append(ingrediente_sel)
                        print(f"➕ Añadido: {ingrediente_sel}")
                    else:
                        print("Ya añadiste ese ingrediente. ¡Concentración!")
                else:
                    print("Opción no válida. Inténtalo de nuevo.")
            else:
                print("Entrada no reconocida. Usa el número o 'S'.")

        # 4. Detener el temporizador (si no se fue antes)
        cliente.atendido.set()
        tiempo_fin = time.time()
        tiempo_tomado = tiempo_fin - tiempo_inicio

        # Esperar a que el hilo del cliente termine (solo por seguridad)
        cliente.join()

        # 5. Verificar el resultado
        self.verificar_pedido(cliente, ingredientes_hechos, tiempo_tomado)

    def verificar_pedido(self, cliente, ingredientes_hechos, tiempo_tomado):
        """Verifica el tiempo y la precisión del pedido."""

        # 1. VERIFICACIÓN DE TIEMPO
        if cliente.se_fue.is_set():
            print(f"❌ FALLO CRÍTICO: {cliente.id} se fue por lento.")
            self.clientes_perdidos += 1
            return

        print(f"\n¡Pizza servida en {tiempo_tomado:.2f} segundos!")

        # 2. VERIFICACIÓN DE INGREDIENTES (Correcto si es un conjunto exacto)
        req_set = set(cliente.ingredientes_req)
        hecho_set = set(ingredientes_hechos)

        es_correcta = req_set == hecho_set

        if not es_correcta:
            faltan = req_set - hecho_set
            sobran = hecho_set - req_set
            print(f"❌ FALLO DE PIZZA: La pizza está mal hecha.")
            if faltan: print(f"  - Faltó: {', '.join(faltan)}")
            if sobran: print(f"  - Sobró: {', '.join(sobran)}")
            selfos.clientes_perdidos += 1
            return

        # 3. VERIFICACIÓN DE PAGO (Grupos 2 y 4)
        if cliente.es_especial:
            # Grupo 2: Evento especial
            print(f"🛑 EVENTO ESPECIAL (G2): Pizza correcta, pero el cliente NO PAGA. Ganancia: $0.")
            self.clientes_satisfechos += 1
        else:
            # Grupo 4: Pasarela de pagos
            precio = len(cliente.ingredientes_req) * 3.5 + 5.0

            # 10% de probabilidad de fallo de pasarela
            if random.random() < 0.10:
                print(f"❌ PASARELA DE PAGOS (G4): El pago de ${precio:.2f} FALLÓ. Ganancia: $0.")
            else:
                self.balance_caja += precio
                print(f"💰 PAGO EXITOSO (G4): Ganancia: ${precio:.2f}. Nuevo Balance: ${self.balance_caja:.2f}")

            self.clientes_satisfechos += 1

        print(f"✅ ¡{cliente.id} SATISFECHO!")

    def reporte_final(self):
        print("\n" + "="*50)
        print("🍕 REPORTE DE CIERRE DE LA PIZZERÍA 💰")
        print("="*50)
        print(f"Clientes Atendidos Correctamente: {self.clientes_satisfechos}")
        print(f"Clientes Perdidos (Tiempo o Error): {self.clientes_perdidos}")
        print(f"Balance Final en Caja: ${self.balance_caja:.2f}")
        print("="*50)


# ==============================================================================
#                      🕹️ JUEGO PRINCIPAL
# ==============================================================================

if __name__ == "__main__":
    # Parámetros iniciales del juego
    JORNADA_CLIENTES = random.randint(5, 8) # Número de rondas
    CLAN_INFECTADO = random.choice([True, False]) # Grupo 5

    juego = Pizzeria(CLAN_INFECTADO)

    print("--- INICIO DE LA JORNADA INTERACTIVA ---")
    print(f"Rondas a Jugar: {JORNADA_CLIENTES}")
    print("OBJETIVO: Igualar los ingredientes y ser rápido.")

    for i in range(JORNADA_CLIENTES):
        print(f"\n==================== RONDA {i + 1} ====================")

        cliente_actual = juego.generar_pedido()
        juego.atender_cliente(cliente_actual)

        # Pequeña pausa entre clientes
        time.sleep(1)

    juego.reporte_final()

⚠️ CLAN INFECTADO (Grupo 5): ¡La tolerancia de los clientes es baja!
--- INICIO DE LA JORNADA INTERACTIVA ---
Rondas a Jugar: 6
OBJETIVO: Igualar los ingredientes y ser rápido.

==================== RONDA 1 ====================

##################################################
| LLEGA: Cliente_1 | CLIENTE NORMAL
| TOLERANCIA: 11.76 segundos
| QUIERE: Mozzarella, Champiñones, Piña, Tomate
##################################################

--- MENÚ DE INGREDIENTES (Elige el número) ---
[1] Tomate
[2] Mozzarella
[3] Pepperoni
[4] Champiñones
[5] Cebolla
[6] Pimiento
[7] Aceitunas
[8] Anchoas
[9] Piña
[S] ¡Servir Pizza! (Terminar pedido)
-----------------------------------
🍕 Pizza actual: Vacía
⏰ TIEMPO RESTANTE: 11.76 segundos

🚨 Cliente_1 ABANDONA: ¡Excediste su tolerancia de 11.76 segundos!
>>> Elige Ingrediente o [S]ervir: 1
➕ Añadido: Tomate
❌ FALLO CRÍTICO: Cliente_1 se fue por lento.

==================== RONDA 2 ====================

#############################################